In [498]:
import time
import pandas as pd
import re
from typing import Dict
import citation_normalisation as cn
import reference_parser as rp
import importlib
importlib.reload(rp)
importlib.reload(cn)

<module 'citation_normalisation' from 'C:\\Users\\Otto Brinkhaus\\Documents\\Arbeit\\COCONUT\\citation_normalisation\\citation_normalisation.py'>

In [479]:
# COCONUT examples
test_list = [
    #'10.7164/antibiotics.33.463',
    #'10.1186/s13321-020-00465-0',
    'Kluepfel,J. Antibiotics 25,(1972),109',
    #'21939219'
    #'1293891283912839128391283912893'
    #'Harborne, The Handbook of Natural Flavonoids, 2, (1999), 517,Isoflavonoids and neoflavonoids'
    #'Ratnayake,J.Nat.Prod.,71,(2008),403',
    #'Probst,Helvetica Chimica Acta,64,(1981),2056',
    #'Braz Filho,Phytochem.,19,(1980),2003'
]
references = []
for test_ID in test_list:
    reference = cn.get_structured_reference(test_ID)
    references.append(reference)
    

In [480]:
# Print exemplary dict
for key in references[0].keys():
    print(key + ' - ' + str(references[0][key]))

indexed - {'date-parts': [[2021, 4, 10]], 'date-time': '2021-04-10T04:12:49Z', 'timestamp': 1618027969764}
reference-count - 0
publisher - Japan Antibiotics Research Association
issue - 2
content-domain - {'domain': [], 'crossmark-restriction': False}
short-container-title - ['J. Antibiot.']
published-print - {'date-parts': [[1972]]}
DOI - 10.7164/antibiotics.25.109
type - journal-article
created - {'date-parts': [[2012, 3, 1]], 'date-time': '2012-03-01T03:25:38Z', 'timestamp': 1330572338000}
page - 109-115
source - Crossref
is-referenced-by-count - 133
title - ['MYRIOCIN, A NEW ANTIFUNGAL ANTIBIOTIC FROM MYRIOCOCCUM ALBOMYCES']
prefix - 10.7164
volume - 25
author - [{'given': 'DIETER', 'family': 'KLUEPFEL', 'sequence': 'first', 'affiliation': []}, {'given': 'JEHAN', 'family': 'BAGLI', 'sequence': 'additional', 'affiliation': []}, {'given': 'HAROLD', 'family': 'BAKER', 'sequence': 'additional', 'affiliation': []}, {'given': 'MARIE-PAULE', 'family': 'CHAREST', 'sequence': 'additional', 

In [ ]:
for reference_dict in references:
    reference_str = cn.create_normalized_reference_str(reference_dict)
    print(reference_str)

In [2]:
# Read data from file
coconut_references = pd.read_csv('./coconut_references.csv')
unstructured_references = coconut_references['citationDOI']
COCONUT_IDs = coconut_references['coconut_id']

ID_ref_tuples = zip(COCONUT_IDs, unstructured_references)

# Get everything into the right format and filter empty reference lists
ID_ref_tuples = [(tup[0], eval(tup[1])) # the references are read as str and need to be converted to lists
                for tup in ID_ref_tuples
                if len(eval(tup[1])) != 0 # Don't include empty reference lists
                if eval(tup[1]) != ['NA']] # Don't include reference list with 'NA' as only element


In [459]:
parser = rp.reference_parser()

In [460]:
## Check regex for typical pattern: 
# Pattern: Author,? (et al.)?, Journal, issue, (year), page(-page)?

# Examples that are supposed to be matched
general_pattern_references = [
    'Haba,Phytochem.,68,(2007),1255',
    'El-Sayed,Phytochem.,30,(1991),2442',
    'Fujita,J.Nat.Prod.,49,(1986),1122-1125',
    'Kim, et al., Chem Pharm Bull, 52, (2004), 1466',
    'Lansky et al.,J.Ethnopharmacol.,19,(2007),177-206',
    'Imperato,Chim.Ind.(Milan),71,(1989),86',
    'Cole,R.J.et al.,Can.J.Microbiol.,20(1974),1159',
    'Mathews.,J. Biol. Chem.,241(21),(1966),5008',
    " Fang,Chung Ts'ao Yao,12,(1981),1",
    'N.V.Thu,Pharmazie,26,(1971),504',
    'Ruan,Yun-Nan Chih Wu Yen Chiu,13,(1991),225',
    'Peng J.-P.,Phytochem.,41,(1996),283-285',
    ' Hussain,J.Nat.Prod.51.,(1988),809',
    'Bondarenko,Khim.Prir.Soedin,(1983),243',
    'Haba,Phytochem.,68,82007),1255',
    'Ingham,Phytochem.,15,819769,1489',
    'Bondarenko,Khim.Prir.Soedin,(1983),243',]


# Test
for ref in general_pattern_references:
    assert parser.parse_general_pattern(ref)
    print(' Original reference: {} \n Resulting dict:\n {}\n'.format(ref, parser.parse_general_pattern(ref)))
    
    


 Original reference: Haba,Phytochem.,68,(2007),1255 
 Resulting dict:
 {'authors': 'Haba', 'first_author_surname': 'Haba', 'journal': 'Phytochem.', 'volume': '68', 'issue': None, 'year': '2007', 'pages': '1255', 'first_page': '1255'}

 Original reference: El-Sayed,Phytochem.,30,(1991),2442 
 Resulting dict:
 {'authors': 'El-Sayed', 'first_author_surname': 'El-Sayed', 'journal': 'Phytochem.', 'volume': '30', 'issue': None, 'year': '1991', 'pages': '2442', 'first_page': '2442'}

 Original reference: Fujita,J.Nat.Prod.,49,(1986),1122-1125 
 Resulting dict:
 {'authors': 'Fujita', 'first_author_surname': 'Fujita', 'journal': 'J.Nat.Prod.', 'volume': '49', 'issue': None, 'year': '1986', 'pages': '1122-1125', 'first_page': '1122'}

 Original reference: Kim, et al., Chem Pharm Bull, 52, (2004), 1466 
 Resulting dict:
 {'authors': 'Kim, ', 'first_author_surname': 'Kim', 'journal': 'Chem Pharm Bull', 'volume': '52', 'issue': None, 'year': '2004', 'pages': '1466', 'first_page': '1466'}

 Original

In [461]:
# Test parser for a rarer structured reference pattern
underscore_pattern_references = ['J_Agric_Food_Chem_2016_64_(21):4255-4263',
                                 'J_Nat_Prod_2015_78_(4):730-735',
                                 "Phytochemistry_2003;64:285-291"]

for ref in underscore_pattern_references:
    assert parser.parse_underscore_pattern(ref)
    print(' Original reference: {} \n Resulting dict:\n {}\n'.format(ref, parser.parse_underscore_pattern(ref)))


 Original reference: J_Agric_Food_Chem_2016_64_(21):4255-4263 
 Resulting dict:
 {'journal': 'J Agric Food Chem', 'year': '2016', 'volume': '64', 'issue': '21', 'pages': '4255-4263', 'first_page': '4255'}

 Original reference: J_Nat_Prod_2015_78_(4):730-735 
 Resulting dict:
 {'journal': 'J Nat Prod', 'year': '2015', 'volume': '78', 'issue': '4', 'pages': '730-735', 'first_page': '730'}

 Original reference: Phytochemistry_2003;64:285-291 
 Resulting dict:
 {'journal': 'Phytochemistry', 'year': '2003', 'volume': '64', 'issue': None, 'pages': '285-291', 'first_page': '285'}



In [462]:
# Regex for J.Chem.Soc. references 

jchemsoc_pattern_references = [
    'Gunasekera,J.Chem.Soc.,Perkin 1,(1975),2447',
    'Locksley,J.Chem.Soc.,C,(1971),1332',]

# Test
for ref in jchemsoc_pattern_references:
    assert parser.parse_jchemsoc_pattern(ref)
    print(' Original reference: {} \n Resulting dict:\n {}\n'.format(ref, parser.parse_jchemsoc_pattern(ref)))

#print(re.search(j_chem_soc_pattern, ref_str_1).groupdict())
#assert re.search(j_chem_soc_pattern, ref_str_1)
#assert re.search(j_chem_soc_pattern, ref_str_2)

 Original reference: Gunasekera,J.Chem.Soc.,Perkin 1,(1975),2447 
 Resulting dict:
 {'authors': 'Gunasekera', 'first_author_surname': 'Gunasekera', 'journal': 'J.Chem.Soc., Perkin 1', 'year': '1975', 'pages': '2447', 'first_page': '2447'}

 Original reference: Locksley,J.Chem.Soc.,C,(1971),1332 
 Resulting dict:
 {'authors': 'Locksley', 'first_author_surname': 'Locksley', 'journal': 'J.Chem.Soc., C', 'year': '1971', 'pages': '1332', 'first_page': '1332'}



In [463]:
# Check regex for the Handbook of Natural Flavonoids (Harborne)

harborne_flavonoid_references = [
    'Harborne, The Handbook of Natural Flavonoids, 2, (1999), 115,Chalcones,dihydrochalcones and aurones',
    'Harborne, The Handbook of Natural Flavonoids, 1, (1999), 181.Flavonols',
    'Harborne, The Handbook of Natural Flavonoids, 1, (1999), 3.Flavone O-glycosides, John Wiley & Son',]

for ref in harborne_flavonoid_references:
    assert parser.parse_harborne_flavonoid_pattern(ref)
    print(' Original reference: {} \n Resulting dict:\n {}\n'.format(ref, parser.parse_harborne_flavonoid_pattern(ref)))

 Original reference: Harborne, The Handbook of Natural Flavonoids, 2, (1999), 115,Chalcones,dihydrochalcones and aurones 
 Resulting dict:
 {'authors': 'Harborne, J.B., Baxter, H.', 'title': 'The Handbook of Natural Flavonoids', 'volume': '2', 'year': '1999', 'chapter_no': '115', 'chapter_title': 'Chalcones,dihydrochalcones and aurones', 'publisher': 'Wiley', 'doi': '10.1016/S0039-9140(00)00629-9', 'isbn': '0-471-95893-2'}

 Original reference: Harborne, The Handbook of Natural Flavonoids, 1, (1999), 181.Flavonols 
 Resulting dict:
 {'authors': 'Harborne, J.B., Baxter, H.', 'title': 'The Handbook of Natural Flavonoids', 'volume': '1', 'year': '1999', 'chapter_no': '181', 'chapter_title': 'Flavonols', 'publisher': 'Wiley', 'doi': '10.1016/S0039-9140(00)00629-9', 'isbn': '0-471-95893-1'}

 Original reference: Harborne, The Handbook of Natural Flavonoids, 1, (1999), 3.Flavone O-glycosides, John Wiley & Son 
 Resulting dict:
 {'authors': 'Harborne, J.B., Baxter, H.', 'title': 'The Handbook

In [464]:
# Check regex for the Phytochemical Dictionary (Harborne)

harborne_phytochemdict_references = [
    'Harborne,Phytochemical Dictionary Second Edition,Taylor and Francis,(1999),Chapter54',]



for ref in harborne_phytochemdict_references:
    assert parser.parse_harborne_phytochemdict_pattern(ref)
    print(' Original reference: {} \n Resulting dict:\n {}\n'.format(ref, parser.parse_harborne_phytochemdict_pattern(ref)))

 Original reference: Harborne,Phytochemical Dictionary Second Edition,Taylor and Francis,(1999),Chapter54 
 Resulting dict:
 {'year': '1999', 'chapter_no': '54', 'authors': 'Harborne, J.B., Baxter, H., Moss, G.P.', 'publisher': 'Taylor & Francis', 'title': 'Phytochemical Dictionary. A Handbook of Bioactive Compounds from Plants (Second Edition)', 'doi': 'https://doi.org/10.4324/9780203483756', 'isbn': '9780748406203'}



In [465]:
# How many references are there and what type of references are we dealing with?

total_number = 0 # Total number of references
unique_number = 0 # Tumber of unique references
PMID_number = 0 # Number of PubMed IDs
DOI_number = 0 # Number of DOIs
general_pattern_number = 0 # Number of references that can be matched exactly be the pattern above.

harborne_flavonoid_number = 0
harborne_phytochemdict_number = 0
underscore_pattern_number = 0
j_chem_soc_number = 0
no_digits_number = 0
suspiciously_short_number = 0
unmatched_references = []
no_digits_references = []
suspiciously_short_references = []


references = []
for tup in ID_ref_tuples:
    for ref in tup[1]:
        if ref != "NA":
            total_number += 1
            if ref not in references:
                unique_number += 1
                references.append(ref)
                # Check for DOI
                if cn.contains_DOI(ref):
                    DOI_number += 1
                # Check for PMID (reference str is a number of at least 6 digits)
                elif ref.isdigit():
                    if len(ref) > 3:
                        PMID_number += 1
                # Check for the most frequent reference notation pattern
                elif parser.parse_general_pattern(ref):
                    general_pattern_number += 1
                # Check for other reference notation patterns
                elif parser.parse_underscore_pattern(ref):
                    underscore_pattern_number += 1
                elif parser.parse_jchemsoc_pattern(ref):
                    j_chem_soc_number += 1
                # Check for Harbornes Handbook of Natural Flavonoids
                elif parser.parse_harborne_flavonoid_pattern(ref):
                    harborne_flavonoid_number += 1
                # Check for Harbornes Phytochemical Dictionary Second Edition
                elif parser.parse_harborne_phytochemdict_pattern(ref):
                    harborne_phytochemdict_number += 1
                # Check for (useless) references that contain no digits
                elif re.search('\D+', ref).group() == ref:
                    no_digits_number += 1
                    no_digits_references.append(ref)
                elif len(ref) < 10:
                    suspiciously_short_number += 1
                    suspiciously_short_references.append(ref)
                else:
                    unmatched_references.append((tup[0], ref))


In [466]:
non_specific_ref = unique_number - DOI_number - PMID_number - general_pattern_number - harborne_flavonoid_number - harborne_phytochemdict_number - underscore_pattern_number - j_chem_soc_number

print('There are {} COCONUT entries'.format(len(COCONUT_IDs)))
print('{} of them have a total of {} references ({} of them are unique).'.format(len(ID_ref_tuples), total_number, unique_number))
print('{} of them are PMIDs.'.format(PMID_number))
print('{} of them are DOIs.'.format(DOI_number))
print('Another {} of them follow a very specific pattern (as in: Haba,Phytochem.,68,(2007),1255). This would offer enough information'.format(general_pattern_number))
print('Another {} of them follow a different specific pattern (as in: J_Nat_Prod_2015_78_(4):730-735).'.format(underscore_pattern_number))
print('Another {} of them follow a different pattern that only occures in J Chem Soc references (as in: Locksley,J.Chem.Soc.,C,(1971),1332)'.format(j_chem_soc_number))
print('Another {} of them come from Harborne´s Handbook of Natural Flavonoids'.format(harborne_flavonoid_number))
print('Another {} of them come from Harborne´s Phytochemical Dictionary Second Edition'.format(harborne_phytochemdict_number))
print('That leaves us with {} unique references that do not match a specific pattern (that we know).'.format(non_specific_ref))
print('{} of them do not contain any digit.'.format(no_digits_number))
print('{} of the remaining references are shorter than 10 characters.'.format(suspiciously_short_number))


There are 406747 COCONUT entries
70969 of them have a total of 158824 references (66151 of them are unique).
20953 of them are PMIDs.
12232 of them are DOIs.
Another 29297 of them follow a very specific pattern (as in: Haba,Phytochem.,68,(2007),1255). This would offer enough information
Another 64 of them follow a different specific pattern (as in: J_Nat_Prod_2015_78_(4):730-735).
Another 111 of them follow a different pattern that only occures in J Chem Soc references (as in: Locksley,J.Chem.Soc.,C,(1971),1332)
Another 589 of them come from Harborne´s Handbook of Natural Flavonoids
Another 59 of them come from Harborne´s Phytochemical Dictionary Second Edition
That leaves us with 2846 unique references that do not match a specific pattern (that we know).
285 of them do not contain any digit.
155 of the remaining references are shorter than 10 characters.


In [467]:
for ref in unmatched_references[:150]:
    print(ref)

('CNP0106606', 'Cole,Handbook of Secondary Fungal Metabolites,Volume I,(2003)')
('CNP0309481', 'Yin, et al., Modern Study of Chinese Drugs and Clinical Applications (1), Xueyuan Press, Beijing, (1993).')
('CNP0309481', 'Buckingham(Executive Editor), Dictionary of Natural Products, Chapman & Hall, 1994, Vol1-7')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1995, Vol8')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1996, Vol9')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1997, Vol10')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1998, Vol11.')
('CNP0251815', 'Ohmiya,The Alkaloids,47,(1995),1,Lupine alkaloids')
('CNP0251815', 'Ji, et al., Pharmacological Action and Application of Available Composition of Traditional Chinese Medicine, Heilongjiang Science and technology Press, Heilongjiang, (1995).')
('CNP0251815', '

## Match parsed information and retrieved information

In [468]:
parsed_ref_data = parser('Kluepfel,J. Antibiotics 25,(1972),109')
parsed_ref_data

{'authors': 'Kluepfel',
 'first_author_surname': 'Kluepfel',
 'journal': 'J. Antibiotics ',
 'volume': '25',
 'issue': None,
 'year': '1972',
 'pages': '109',
 'first_page': '109'}

In [500]:
retrieved_ref_data = cn.get_structured_reference('Kluepfel,J. Antibiotics 25,(1972),109')

In [499]:
norm_retrieved_ref_data = cn.normalize_crossref_dict(retrieved_ref_data)

{'title': 'MYRIOCIN, A NEW ANTIFUNGAL ANTIBIOTIC FROM MYRIOCOCCUM ALBOMYCES',
 'DOI': '10.7164/antibiotics.25.109',
 'issue': '2',
 'volume': '25',
 'year': 1972,
 'journal': 'The Journal of Antibiotics',
 'authors': ['Kluepfel, D.',
  'Bagli, J.',
  'Baker, H.',
  'Charest, M.',
  'Kudelski, A.',
  'Sehgal, S., N.',
  'Vézina, C.'],
 'page': '109-115'}

In [471]:
def is_same_publication(parsed_ref_data: Dict, retrieved_ref_data: Dict) -> bool:
    '''
    This function compares two dictionaries with parsed and retrieved reference data and
    return True if all of the following are the same:
    - year
    - family name of first author
    - issue (if given)
    - volume
    - first page
    '''
    pass

In [495]:
retrieved_ref_data

{'indexed': {'date-parts': [[2021, 4, 10]],
  'date-time': '2021-04-10T04:12:49Z',
  'timestamp': 1618027969764},
 'reference-count': 0,
 'publisher': 'Japan Antibiotics Research Association',
 'issue': '2',
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'short-container-title': ['J. Antibiot.'],
 'published-print': {'date-parts': [[1972]]},
 'DOI': '10.7164/antibiotics.25.109',
 'type': 'journal-article',
 'created': {'date-parts': [[2012, 3, 1]],
  'date-time': '2012-03-01T03:25:38Z',
  'timestamp': 1330572338000},
 'page': '109-115',
 'source': 'Crossref',
 'is-referenced-by-count': 133,
 'title': ['MYRIOCIN, A NEW ANTIFUNGAL ANTIBIOTIC FROM MYRIOCOCCUM ALBOMYCES'],
 'prefix': '10.7164',
 'volume': '25',
 'author': [{'given': 'DIETER',
   'family': 'KLUEPFEL',
   'sequence': 'first',
   'affiliation': []},
  {'given': 'JEHAN',
   'family': 'BAGLI',
   'sequence': 'additional',
   'affiliation': []},
  {'given': 'HAROLD',
   'family': 'BAKER',
   'sequence': 'add